<a href="https://colab.research.google.com/github/nadya-58/MIDRC_colab/blob/main/Lung_Volume_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!gcloud config set project  bwh-midrc-rapid-res-1655321320

Updated property [core/project].


In [2]:
from google.colab import auth
auth.authenticate_user()

In [3]:
  project_name = "bwh-midrc-rapid-res-1655321320"
  bucket_name = "midrc-analysis-bwh"
  bucket_path = "bpr-results/"

In [4]:
import os
import shutil
import glob
import json
import matplotlib.pyplot as plt
from matplotlib import transforms
from matplotlib.colors import ListedColormap
import numpy as np

In [ ]:
bucket_name = 'midrc-analysis'
bucket_path = 'bpr-results'
#!gsutil mb gs://{bucket_name}
!gsutil cp /content/json_data/2.16.840.1.114274.1818.56006340868836576058065240047448675238.json gs://{bucket_name}/{bucket_path}
!gsutil cat gs://{bucket_name}/{bucket_path}/2.16.840.1.114274.1818.56006340868836576058065240047448675238.json

CommandException: No URLs matched: /content/json_data/2.16.840.1.114274.1818.56006340868836576058065240047448675238.json


In [ ]:
file_exists_in_bucket(project_name, bucket_name, "gs://midrc-analysis/bpr-results/2.16.840.1.114274.1818.56006340868836576058065240047448675239.json")

Searching gs://midrc-analysis/ for: 
bpr-results/2.16.840.1.114274.1818.56006340868836576058065240047448675239.json



False

In [5]:
!pip install dicomweb-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 58 kB 3.2 MB/s 
     |████████████████████████████████| 2.0 MB 10.8 MB/s 
     |████████████████████████████████| 3.1 MB 34.8 MB/s 
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11447 sha256=cb95a5bb5c1deb2df91a107801f099fa393a0feeea3f5664e5d1100863b7c60a
  Stored in directory: /root/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying
  Attempting uninstall: Pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2


In [6]:
#install nrrd
!pip install pynrrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
!pip install SimpleITK

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 52.8 MB 221 kB/s 


In [8]:
#Install Plastimatch

!sudo apt install plastimatch 
!echo $(plastimatch --version)

if os.path.isdir('/content/pyplastimatch'):
  try:
    shutil.rmtree('/content/pyplastimatch')
  except OSError as err:
    print("Error: %s : %s" % ("pyplastimatch", err.strerror)) 
# !git clone https://github.com/denbonte/pyplastimatch/ pyplastimatch
!git clone https://github.com/AIM-Harvard/pyplastimatch.git 

# from pyplastimatch import pyplastimatch as pypla
from pyplastimatch.pyplastimatch import pyplastimatch as pypla

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libdcmtk12 libdlib-data libdlib18 libfftw3-single3 libinsighttoolkit4.12
  libnifti2
Suggested packages:
  libfftw3-bin libfftw3-dev
The following NEW packages will be installed:
  libdcmtk12 libdlib-data libdlib18 libfftw3-single3 libinsighttoolkit4.12
  libnifti2 plastimatch
0 upgraded, 7 newly installed, 0 to remove and 20 not upgraded.
Need to get 76.0 MB of archives.
After this operation, 162 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libdcmtk12 amd64 3.6.2-3build3 [4,499 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libdlib-data all 18.18-2build1 [63.4 MB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 

In [9]:
!pip install git+https://github.com/JoHof/lungmask

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/JoHof/lungmask to /tmp/pip-req-build-hbjqgiyz
  Running command git clone -q https://github.com/JoHof/lungmask /tmp/pip-req-build-hbjqgiyz
     |████████████████████████████████| 1.1 MB 5.2 MB/s 
     |████████████████████████████████| 4.2 MB 31.7 MB/s 
  Created wheel for lungmask: filename=lungmask-0.2.11-py3-none-any.whl size=17456 sha256=bc93974e6aba7677e35ceb09cf31c3f9c8481da22b5059e22a888f01ca67a984
  Stored in directory: /tmp/pip-ephem-wheel-cache-0ctivuz8/wheels/b7/62/f7/fd172f57224a95cf73efec4a6029c8a78577dab75b949338d1
Successfully built lungmask


In [10]:
%%bigquery --project=bwh-midrc-rapid-res-1655321320 ct_limited_open_a1_r1

WITH
  nlst_instances_per_series AS (
    SELECT
      DISTINCT(StudyInstanceUID),
      SeriesInstanceUID,
      COUNT(DISTINCT(SOPInstanceUID)) AS num_instances,
      COUNT(DISTINCT(ARRAY_TO_STRING(ImagePositionPatient,"/"))) AS position_count,
      COUNT(DISTINCT(ARRAY_TO_STRING(ImageOrientationPatient,"/"))) AS orientation_count,
      MIN(SAFE_CAST(SliceThickness AS float64)) AS min_SliceThickness,
      MAX(SAFE_CAST(SliceThickness AS float64)) AS max_SliceThickness,
      MIN(SAFE_CAST(ImagePositionPatient[SAFE_OFFSET(2)] AS float64)) as min_SliceLocation, 
      MAX(SAFE_CAST(ImagePositionPatient[SAFE_OFFSET(2)] AS float64)) as max_SliceLocation,
      STRING_AGG(DISTINCT(SAFE_CAST("LOCALIZER" IN UNNEST(ImageType) AS string)),"") AS has_localizer
    FROM
      bwh-midrc-rapid-res-1655321320.midrc_dicom_us.dicom_all
    WHERE
      (collection_id = "Open-R1" or collection_id = "Open-A1") and Modality = "CT"
    GROUP BY
      StudyInstanceUID,
      SeriesInstanceUID
      ), 
  nlst_values_per_series AS (
    SELECT 
    dicom_all.StudyInstanceUID AS StudyInstanceUID,
    ANY_VALUE(dicom_all.PatientID) AS PatientID,
    ANY_VALUE(dicom_all.SeriesInstanceUID) AS SeriesInstanceUID,
    ANY_VALUE(nlst_instances_per_series.num_instances) AS num_instances,
    ANY_VALUE(nlst_instances_per_series.max_SliceThickness) AS SliceThickness,
    ANY_VALUE((nlst_instances_per_series.max_SliceLocation - nlst_instances_per_series.min_SliceLocation)) AS PatientHeightScanned
  FROM
    bwh-midrc-rapid-res-1655321320.midrc_dicom_us.dicom_all AS dicom_all
  JOIN
    nlst_instances_per_series
  ON
    dicom_all.SeriesInstanceUID = nlst_instances_per_series.SeriesInstanceUID
  WHERE
    min_SliceThickness >= 1.5 
    AND max_SliceThickness <= 3.5 
    AND nlst_instances_per_series.num_instances > 100
    AND nlst_instances_per_series.num_instances/nlst_instances_per_series.position_count = 1
    AND nlst_instances_per_series.orientation_count = 1
    AND has_localizer = "false"
  GROUP BY
    StudyInstanceUID
  )
  SELECT 
    dicom_all.PatientID,
    dicom_all.StudyInstanceUID,
    dicom_all.SeriesInstanceUID,
    dicom_all.SOPInstanceUID,
    dicom_all.collection_id,
    dicom_all.PatientAge,
    dicom_all.PatientWeight,
    nlst_values_per_series.num_instances,
    nlst_values_per_series.SliceThickness,
    nlst_values_per_series.PatientHeightScanned
  FROM
    bwh-midrc-rapid-res-1655321320.midrc_dicom_us.dicom_all AS dicom_all
  JOIN
    nlst_values_per_series 
  ON
    dicom_all.SeriesInstanceUID = nlst_values_per_series.SeriesInstanceUID

In [11]:
from google.cloud import storage

storage_client = storage.Client(project = project_name)
#bucket = storage_client.get_bucket(bucket_name)

series_instance_uids = []

#blobs = storage_client.list_blobs(bucket)
blobs = storage_client.list_blobs(bucket_name, prefix=bucket_path, delimiter='/')
for blob in blobs:
    bn = blob.name
    bs = bn.split('.')
    if bs[-1] == 'json':
      bss = bn.split('/')[1]
      bs3 = bss.split('.')
      bs4 = '.'.join(bs3[:-1])
      series_instance_uids.append(bs4)

print(series_instance_uids[:10])

['1.2.826.0.1.3680043.10.474.419639.105799060738901793068313281334', '1.2.826.0.1.3680043.10.474.419639.106364025147079899289440200715', '1.2.826.0.1.3680043.10.474.419639.149051607502633615235577977952', '1.2.826.0.1.3680043.10.474.419639.189346812051260775638656981947', '1.2.826.0.1.3680043.10.474.419639.192916356998524553834723357563', '1.2.826.0.1.3680043.10.474.419639.198735019931123383691750806063', '1.2.826.0.1.3680043.10.474.419639.249044315484665760654506668895', '1.2.826.0.1.3680043.10.474.419639.259584978733948574762940092562', '1.2.826.0.1.3680043.10.474.419639.269534881585852761235289087419', '1.2.826.0.1.3680043.10.474.419639.272117657178318732150423166273']


In [12]:
print(len(series_instance_uids))

495


In [13]:
df = ct_limited_open_a1_r1
k=0

good_seriesuids = []

#series_instance_uids = ['1.2.826.0.1.3680043.10.474.419639.106364025147079899289440200715']

for seriesuid in series_instance_uids:
  #print(seriesuid)
  studyuids = list(set(df[df['SeriesInstanceUID']==seriesuid]['StudyInstanceUID'].tolist()))
  if len(studyuids)>0:
    k=k+1
    print(seriesuid)
    studyuid = studyuids[0]
    sops = df[ (df['StudyInstanceUID']==studyuid) & (df['SeriesInstanceUID']==seriesuid) ]['SOPInstanceUID'].tolist()
    print(len(sops))
    #download_series(studyuid, seriesuid, sops, path_downloaded)
    good_seriesuids.append(seriesuid)
print(k)

1.2.826.0.1.3680043.10.474.419639.105799060738901793068313281334
178
1.2.826.0.1.3680043.10.474.419639.106364025147079899289440200715
180
1.2.826.0.1.3680043.10.474.419639.192916356998524553834723357563
111
1.2.826.0.1.3680043.10.474.419639.198735019931123383691750806063
184
1.2.826.0.1.3680043.10.474.419639.269534881585852761235289087419
262
1.2.826.0.1.3680043.10.474.419639.272117657178318732150423166273
194
1.2.826.0.1.3680043.10.474.419639.334475370484806268089381726911
206
1.2.826.0.1.3680043.10.474.419639.336871778486950686179971404477
271
1.2.826.0.1.3680043.10.474.419639.337058107440810621229570649505
194
1.2.826.0.1.3680043.10.474.419639.337142866003655727331878751504
114
1.2.826.0.1.3680043.10.474.419639.939657231423501545181087708673
658
1.2.826.0.1.3680043.10.474.419639.995661493936010249470686852549
106
1.3.6.1.4.1.5962.99.1.1247991691.71736179.1616155726490.11471.0
997
1.3.6.1.4.1.5962.99.1.1247991691.71736179.1616155726490.1276.0
209
1.3.6.1.4.1.5962.99.1.1247991691.7173

In [ ]:
path_downloaded = '/content/downloaded_data'
#path_nifti = '/content/nifti_data'
#path_json =  '/content/json_data'
path_nrrd = '/content/nrrd_data'

for x in [path_downloaded, path_nifti, path_json, path_nrrd]:
  if os.path.isdir(x):
    try:
      shutil.rmtree(x)
    except OSError as err:
      print("Error: %s : %s" % (x, err.strerror))  
  os.mkdir(x)

In [ ]:
from lungmask import mask
import SimpleITK as sitk

input_image = sitk.ReadImage(/content/sorted_data/10000364-1546758/CT)

In [ ]:
import json

#for pat in pat_list:
#  pat_dir_path_json = os.path.join(json_directory, pat)
  
#  json_list = glob(os.path.join( pat_dir_path_json,'*.json'))
#  print(len(json_list), 'files for this patient')

jn = "/content/2.16.840.1.114274.1818.504395315714442999311621620775113246374.json"
fileObject = open(jn, "r")
jsonContent = fileObject.read()
aList = json.loads(jsonContent)

print(aList['body part examined'])

jj = os.path.split(jn)
jjp = jj[0] # path
jjn = jj[1] # json file name
bpfn = os.path.join(jjp, jjn.split('.')[0]+'_bodypart_regr.txt')

f = open(bpfn,'w')
for (bodypart, zlist) in aList['body part examined'].items():
  #print(jn)
  print(bodypart, zlist)
  if (len(zlist)>0):
    f.write(bodypart+'\n')
f.close()


{'legs': [], 'pelvis': [], 'abdomen': [295.0, 296.0, 297.0, 298.0, 299.0, 300.0, 301.0, 302.0, 303.0, 304.0, 305.0, 306.0, 307.0, 308.0, 309.0, 310.0, 311.0, 312.0, 313.0, 314.0, 315.0, 316.0, 317.0, 318.0, 319.0, 320.0, 321.0, 322.0, 323.0, 324.0, 325.0, 326.0, 327.0, 328.0, 329.0, 330.0, 331.0, 332.0, 333.0, 334.0, 335.0, 336.0, 337.0, 338.0, 339.0, 340.0, 341.0, 342.0, 343.0, 344.0, 345.0, 346.0, 347.0, 348.0, 349.0, 350.0, 351.0, 352.0, 353.0, 354.0, 355.0, 356.0, 357.0, 358.0, 359.0, 360.0, 361.0, 362.0, 363.0, 364.0, 365.0, 366.0, 367.0, 368.0, 369.0, 370.0, 371.0, 372.0, 373.0, 374.0, 375.0, 376.0, 377.0, 378.0, 379.0, 380.0, 381.0, 382.0, 383.0, 384.0, 385.0, 386.0, 387.0, 388.0, 389.0, 390.0, 391.0, 392.0, 393.0, 394.0, 395.0, 396.0, 397.0, 398.0, 399.0, 400.0, 401.0, 402.0, 403.0, 404.0, 405.0, 406.0, 407.0, 408.0, 409.0, 410.0, 411.0, 412.0, 413.0, 414.0, 415.0, 416.0, 417.0, 418.0, 419.0, 420.0, 421.0, 422.0, 423.0, 424.0, 425.0, 426.0, 427.0, 428.0, 429.0, 430.0, 431.0, 43

In [ ]:
!rm -rf /content/downloaded_data//